# **Explorative Datenanalyse für Disposition**

### Import von Bibliotheken und Daten

In [1]:
import pandas as pd
import plotly.express as px
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
aisles = pd.read_csv("Daten/aisles.csv.zip")
departments = pd.read_csv("Daten/departments.csv.zip")
order_products_s = pd.read_csv("Daten/order_products_s.csv.zip")
orders_s = pd.read_csv("Daten/orders_s.csv.zip")
products = pd.read_csv("Daten/products.csv.zip")
tips = pd.read_csv("Daten/tips.csv.zip")

### Auffälligkeiten in den Daten

#### "Missing" als Aisles 

In [2]:
aisles["aisle"].unique()

array(['prepared soups salads', 'specialty cheeses',
       'energy granola bars', 'instant foods',
       'marinades meat preparation', 'other', 'packaged meat',
       'bakery desserts', 'pasta sauce', 'kitchen supplies',
       'cold flu allergy', 'fresh pasta', 'prepared meals',
       'tofu meat alternatives', 'packaged seafood', 'fresh herbs',
       'baking ingredients', 'bulk dried fruits vegetables',
       'oils vinegars', 'oral hygiene', 'packaged cheese', 'hair care',
       'popcorn jerky', 'fresh fruits', 'soap', 'coffee', 'beers coolers',
       'red wines', 'honeys syrups nectars', 'latino foods',
       'refrigerated', 'packaged produce', 'kosher foods',
       'frozen meat seafood', 'poultry counter', 'butter',
       'ice cream ice', 'frozen meals', 'seafood counter',
       'dog food care', 'cat food care', 'frozen vegan vegetarian',
       'buns rolls', 'eye ear care', 'candy chocolate', 'mint gum',
       'vitamins supplements', 'breakfast bars pastries',
       '

#### "Missing" als Department

In [3]:
departments

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


#### NaN Werte in der "days_since_prior_order" Spalte

In [4]:
orders_s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1673021 entries, 0 to 1673020
Data columns (total 6 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   order_id                1673021 non-null  int64  
 1   user_id                 1673021 non-null  int64  
 2   order_number            1673021 non-null  int64  
 3   order_dow               1673021 non-null  int64  
 4   order_hour_of_day       1673021 non-null  int64  
 5   days_since_prior_order  1569917 non-null  float64
dtypes: float64(1), int64(5)
memory usage: 76.6 MB


In [5]:
orders_s[orders_s["days_since_prior_order"].isna()]["order_number"].unique()

array([1], dtype=int64)

&rarr; Das lässt sich dadurch erklären, dass dies die ersten Bestellungen eines Benutzers sind

Jetzt entfernen wir die Spalte "Unnamed: 0", da sie keine relevanten Informationen enthält.

In [6]:
tips = tips.drop(columns="Unnamed: 0")
tips

,order_id,tip
0,2539329,False
1,2398795,False
2,473747,False
3,2254736,False
4,431534,False
...,...,...
3346078,2585586,False
3346079,943915,True
3346080,2371631,False
3346081,1716008,False


Wir wandeln die Spalte „tip“ von True/False in 1/0 um, um zukünftige Berechnungen zu erleichtern.

In [7]:
tips['tip'] = tips['tip'].astype(int)

In den Tabellen products und order_products gab es keine Auffälligkeiten

# **Abteilung: Disposition**

In [8]:
df = pd.merge(orders_s, tips, on='order_id', how='left')
df.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip
0,2539329,1,1,2,8,NaN,0
1,2398795,1,2,3,7,15.0,0
2,473747,1,3,3,12,21.0,0
3,2254736,1,4,4,7,29.0,0
4,431534,1,5,4,15,28.0,0


### Prozentsatz der Bestellungen mit Trinkgeld  

In [9]:
df.tip.mean()
df_tips = df[['tip']]

proportion_of_ones = df_tips['tip'].mean()
proportion_of_zeros = 1 - proportion_of_ones

# Dataframe für die Visualisierung
pie_data = pd.DataFrame({
    'value': ['Ja', 'Nein'],
    'proportion': [proportion_of_ones, proportion_of_zeros]
})

# Pie chart 
fig = px.pie(
    pie_data,
    names='value',
    values='proportion',
    title='Anteil der Bestellungen mit und ohne Trinkgeld<br><sub>Bei mehr als der Hälfte der Bestellungen wurde kein Trinkgeld gegeben.',
    color='value',  
    color_discrete_map= {'Ja': '#05cb94', 'Nein': '#e51f30'}  
)

# Legende für die Farben
fig.update_layout(
    legend_title_text="Trinkgeld gegeben?",
)

fig.show()

### Top Kunden

In [10]:
# Gruppieren der Daten nach 'user_id'
user_stats = df.groupby('user_id').agg(
    mean_tip=('tip', 'mean'),  # Durchschnittlicher Trinkgeldwert pro Nutzer
    max_order_number=('order_number', 'max')  # Höchste Bestellnummer pro Nutzer
).reset_index() 

In [11]:
# Sortieren nach durchschnittlichem Trinkgeld und maximaler Bestellnummer
user_stats_top = user_stats.sort_values(by=['mean_tip', 'max_order_number'], ascending=[False, False])

In [12]:
user_stats_top.head(20)

,user_id,mean_tip,max_order_number
6165,12102,1.0,100
8439,16600,1.0,100
32291,64320,1.0,100
33874,67440,1.0,100
39648,79060,1.0,100
46786,93140,1.0,100
62088,123746,1.0,100
76608,152877,1.0,100
82563,164918,1.0,100
90553,180963,1.0,100


### Flop Kunden

In [13]:
# Absteigend nach 'mean_tip' und aufsteigend nach 'max_order_number' sortieren
user_stats_flop = user_stats.sort_values(by=['mean_tip', 'max_order_number'], ascending=[False, True])

In [14]:
user_stats_flop.tail(10)

,user_id,mean_tip,max_order_number
60575,120840,0.0,100
67325,134144,0.0,100
69777,139038,0.0,100
75036,149680,0.0,100
79622,158972,0.0,100
80765,161200,0.0,100
87571,174923,0.0,100
95226,190456,0.0,100
97169,194449,0.0,100
102201,204455,0.0,100


### Zusammenhang zwischen dem Trinkgeldverhalten und der Tageszeit

In [15]:
# Daten nach Stunde gruppieren und den durchschnittlichen Trinkgeldwert (0 oder 1) berechnen,
# um den prozentualen Anteil der Bestellungen mit Trinkgeld zu erhalten
tip_percentage_by_hour = df.groupby('order_hour_of_day')['tip'].mean().reset_index()

tip_percentage_by_hour.columns = ['order_hour_of_day', 'tip_percentage'] # Spalten umbenennen für bessere Lesbarkeit

# Balkendiagramm erstellen
fig = px.bar(
    tip_percentage_by_hour,
    x='order_hour_of_day',
    y='tip_percentage', 
    title="Anteil des Trinkgelds nach Tageszeit<br><sub>Von 5 bis 18 Uhr ist der Anteil an Trinkgeld geringer und steigt am Abend an",
    labels={'order_hour_of_day': 'Tageszeit (Stunde)', 'tip_percentage': 'Trinkgeldanteil'},
    height=500, # Höhe des Diagramms
    color_discrete_sequence=['#05cb94']
)
# Y-Achse auf den Bereich 0 bis 1 begrenzen, da es um Prozentwerte geht
fig.update_layout(yaxis=dict(range=[0,1]))
fig.show()

### Anzahl der Bestellungen nach Tageszeit

In [16]:
# Daten nach Stunde des Tages gruppieren und die Bestellungen summieren
tip_count_by_hour = df.groupby('order_hour_of_day')['order_id'].count().reset_index()
# Spalten für bessere Lesbarkeit umbenennen
tip_count_by_hour.columns = ['order_hour_of_day', 'order_count']

# Balkendiagramm erstellen
fig = px.bar(
    tip_count_by_hour,
    x='order_hour_of_day', 
    y='order_count',
    title="Anzahl der Bestellungen nach Tageszeit<br><sub>Da wo hohes anteil an Trinkgeld ist, gibt es auch viel weniger Bestellungen",
    labels={'order_hour_of_day': 'Tageszeit (Stunde)', 'order_count': 'Anzahl der Bestellungen'},
    height=600, # Höhe des Diagramms
    color_discrete_sequence=['#05cb94']
)

fig.show()

### Trinkgeldverhalten nach Wochentag

In [17]:
# Daten nach Wochentag gruppieren und den Durchschnittswert der Trinkgelder berechnen
tip_count_by_dow = df.groupby('order_dow')['tip'].mean().reset_index()
# Spalten umbenennen für bessere Lesbarkeit
tip_count_by_dow.columns = ['order_dow', 'tip_count']

# Balkendiagramm erstellen
fig = px.bar(
    tip_count_by_dow,
    x='order_dow', 
    y='tip_count',
    title="Trinkgeldverhalten nach Wochentag<br><sub>Der Trinkgeldanteil ist an den Tagen 0 und 1 am höchsten.",
    labels={'order_dow': 'Wochentag', 'tip_count': 'Trinkgeldanteil'},
    height= 600, # Diagrammhöhe
    color_discrete_sequence=['#05cb94'] # Balkenfarbe
)
# Y-Achse auf den Bereich 0 bis 1 begrenzen
fig.update_layout(yaxis=dict(range=[0,1]))
fig.show()

### Trinkgeldverhalten nach Wochentag und Tageszeit

In [18]:
# Daten nach Wochentag und Tageszeit gruppieren und den Durchschnittswert der Trinkgelder berechnen
tip_by_dow_hour = df.groupby(['order_dow', 'order_hour_of_day'])['tip'].mean().reset_index()

# Heatmap erstellen
fig = px.imshow(
    tip_by_dow_hour.pivot(index='order_hour_of_day', columns='order_dow', values='tip'),
    labels=dict(x='Wochentag', y='Tageszeit', color='Anteil'),
    title='Trinkgeldverhalten nach Tageszeit und Wochentag<br><sub>Die Trinkgeldanteil ist abends und nachts höher, besonders an den Wochentagen 0 und 1.', 
    template="plotly_white",
    height=600,
    aspect='auto', # Automatische Anpassung der Achsenverhältnisse

)

fig.update_yaxes(autorange = False ,range = [0,23])
fig.show()


-------

### Zusammenhang zwischen dem Trinkgeldverhalten und  Departments

In [19]:
tips_order_products = order_products_s.merge(products, on= 'product_id', how= 'inner' )
tips_order_products = tips_order_products.merge(departments, on= 'department_id', how= 'inner' )
tips_order_products = tips_order_products.merge(tips, on= 'order_id', how= 'inner' )
tips_order_products = tips_order_products.merge(orders_s, on= 'order_id', how= 'inner' )

In [20]:
# Entfernen der nicht benötigten Spalten aus der Tabelle
tips_order_products = tips_order_products.drop(columns= ['product_id', 'add_to_cart_order', 'product_name', 'aisle_id']) 

In [21]:
# Umwandeln der 'tip'-Spalte von True/False in 1/0
tips_order_products['tip'] = tips_order_products['tip'].astype(int)

In [22]:
# Auswahl relevanter Spalten für die Analyse von Bestellungen nach Abteilung
order_dep_uni = tips_order_products[['order_id', 'department_id','department', 'tip']] 
# Gruppieren nach Abteilung und Berechnung des durchschnittlichen Trinkgeldprozentsatzes
tip_by_department = order_dep_uni.groupby('department')['tip'].mean().reset_index().rename(columns={'tip': 'tip_percentage'}).round(2).sort_values(by='tip_percentage', ascending=False)
# Berechnung des Prozentsatzes von Bestellungen ohne Trinkgeld
tip_by_department['no_tip_percentage'] = 1- tip_by_department['tip_percentage']

In [23]:
# Filtern der Daten: Nur Abteilungen mit einem Trinkgeldanteil größer als 0
filtered_data = tip_by_department[tip_by_department['tip_percentage'] > 0]

# Bestimmen der Abteilung mit dem höchsten Trinkgeldanteil
top_department = filtered_data.iloc[0]  # Erster Eintrag (höchster Trinkgeldanteil)
subtitle = f"Das Department '{top_department['department']}' hat den höchsten Trinkgeldanteil bei Bestellungen"

fig = px.bar(
    filtered_data,
    x='department',  # Abteilung auf der x-Achse
    y='tip_percentage',  # Trinkgeldprozentsatz auf der y-Achse
    title='Trinkgeldverhalten nach Departments',  # Titel des Diagramms
    labels={'department': 'Departments', 'tip_percentage': 'Trinkgeldanteil'},  # Achsenbeschriftungen
    text='tip_percentage',  # Anzeige der Werte auf den Balken
    color = "department",
    color_discrete_sequence=['#05cb94'] + ["lightgrey" for _ in range(len(filtered_data["department"]) - 1)]
)


# Anpassen der Position der Textwerte außerhalb der Balken
fig.update_traces(textposition='outside',showlegend=False)

# Layout-Anpassung: Titel mit Untertitel und Rand
fig.update_layout(
    margin=dict(t=68),  
    title={
        'text': f"Trinkgeldverhalten nach Departments<br><sup>{subtitle}</sup>"  # Titel mit Untertitel
    }
)

fig.update_layout(yaxis=dict(range=[0,1]))


# Diagramm anzeigen
fig.show()


### Trinkgeldverhalten nach Wochentag und Department

In [24]:
# Gruppieren nach Department und Wochentag, dann Berechnung des Trinkgeldanteil
department_tip_percentages = (
    tips_order_products.groupby(["department", "order_dow"])["tip"]
    .mean() 
    .reset_index()  
)

# Umbenennen der Spalte für bessere Lesbarkeit
department_tip_percentages.rename(columns={"tip": "tip_percentage"}, inplace=True)

fig = px.line(
    department_tip_percentages,
    x="order_dow",  # Wochentag auf der x-Achse
    y="tip_percentage",  # Trinkgeldanteil auf der y-Achse
    color="department",  # Linienfarben nach Department
    markers=True,  # Markierungen für Datenpunkte hinzufügen
    labels={"tip_percentage": "Trinkgeldanteil", "order_dow": "Wochentag"},  # Achsenbeschriftungen
    title="Trinkgeldanteil nach Department und Wochentag<br><sub>An den Tagen 0 und 1 haben fast alle Departments einen höheren Trinkgeldanteil im Vergleich zum Rest der Woche.",  # Diagrammtitel
)

# Formatierung der y-Achse als Prozentwerte
fig.update_layout(yaxis=dict(tickformat=".0%", range=[0,1]),xaxis=dict(range=[0,6]))

# Diagramm anzeigen
fig.show()


### Trinkgeldverhalten nach Produktanzahl in einer Bestellung

In [25]:
df_2 = pd.merge(df, order_products_s, on='order_id', how='left')

In [26]:
# Gruppieren der Daten nach 'order_id'
df_agg = df_2.groupby('order_id').agg(
    tip=('tip', 'first'),  # Der erste Trinkgeldwert für jede Bestellung
    product_count=('product_id', 'count')  # Anzahl der Produkte pro Bestellung
).reset_index() 

In [27]:
df_agg.head() #max: 121 Produkte, min 1 Produkt

,order_id,tip,product_count
0,1,0,8
1,2,0,9
2,4,1,13
3,5,1,26
4,8,0,1


In [28]:
# Bins und Labels definieren
bins = [0, 10, 20, 30, 40, 50, 100, float('inf')]  # 6 Bins bis 100, danach 100+
labels = ['1-10', '11-20', '21-30', '31-40', '41-50', '50-100', '100+']  # Beschriftungen

# Produktanzahl in Intervalle einteilen
df_agg['product_count_interval'] = pd.cut(df_agg['product_count'], bins=bins, labels=labels, right=False)

# Gruppieren nach Intervallen und Berechnung der Häufigkeit
grouped = df_agg.groupby('product_count_interval').agg(
    total_orders=('order_id', 'count'),
    tips_given=('tip', 'sum')
).reset_index()

# Berechnung der relativen Häufigkeit
grouped['relative_frequency'] = grouped['tips_given'] / grouped['total_orders']

# Visualisierung der relativen Häufigkeit
fig = px.bar(
    grouped,
    x='product_count_interval',
    y='relative_frequency',
    title='Trinkgeldanteil nach Produktanzahl in einer Bestellung<br><sub>Der Trinkgeldanteil ist bei Bestellungen mit weniger Produkten höher und nimmt mit steigender Produktanzahl ab.',
    labels={'product_count_interval': 'Produktanzahl-Intervall', 'relative_frequency': 'Trinkgeldanteil'},
    text='relative_frequency',
    color_discrete_sequence=['#05cb94']
)

# Layout-Updates
fig.update_traces( textposition='outside', texttemplate='%{text:.2f}')
fig.update_layout(xaxis_title='Produktanzahl', yaxis_title='Trinkgeldanteil', yaxis=dict(range=[0, 1]))
fig.show()

C:\Users\etzko\AppData\Local\Temp\ipykernel_31912\1047500872.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



### Zusammenhang zwischen der Zeit zwischen Bestellungen und Trinkgeld 

In [29]:
df["days_since_prior_order"].max()

30.0

In [30]:
# Durchschnittliche Trinkgeldrate für jeden Wert von 'days_since_prior_order' berechnen
tip_by_days = df.groupby('days_since_prior_order')['tip'].mean().reset_index()

# Liniendiagramm erstellen
px.line(tip_by_days,
        x='days_since_prior_order', 
        y='tip',
        title='Zusammenhang zwischen der Zeit zwischen den Bestellungen und dem Trinkgeld<br><sub>Der Trinkgeldanteil erreicht seinen Höhepunkt nach 7 Tagen seit der letzten Bestellung und sinkt danach wieder.',
        labels={'days_since_prior_order': 'Tage seit der letzten Bestellung',
                'tip': 'Trinkgeldanteil'},
        markers=True, # Marker hinzufügen, um einzelne Punkte sichtbar zu machen
        color_discrete_sequence=["#646cfc"]) 

### Trinkgeldverhalten nach Bestellnummer

Wir untersuchen, wie sich das Trinkgeldverhalten je nach Bestellnummer entwickelt. Zum Beispiel, ob Kunden in späteren Bestellungen häufiger Trinkgeld geben.

In [31]:
# Durchschnittliche Trinkgeldrate für jede Bestellnummer berechnen (z. B. 1. Bestellung, 2. Bestellung usw.)
order_number_df = df.groupby('order_number')['tip'].mean().reset_index()

# Liniendiagramm erstellen
fig = px.line(order_number_df, x='order_number', y='tip', 
              title='Trinkgeldanteil nach Bestellnummer<br><sub>Der Trinkgeldanteil ist am niedrigsten bei neuen Kunden und zwischen der 20. und 58. Bestellung liegt sie über 50%.', 
              labels={'order_number': 'Bestellnummer', 'tip': 'Trinkgeldanteil'}, # Achsenbeschriftungen
              markers=True, #Marker, um einzelne Datenpunkte hervorzuheben
              color_discrete_sequence=["#646cfc"]) # Farbcode für die Linie

fig.add_hline(y=0.5, line_color="red")

fig.show()